In [ ]:
import numpy as np
import pandas as pd
import shap
from sklearn.model_selection import train_test_split, KFold
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, confusion_matrix, cohen_kappa_score
from imblearn.over_sampling import SMOTE
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.models import load_model

In [ ]:
print(tf.__version__)

In [ ]:
# Load the data
df1 = pd.read_csv("sampled_10_percent.csv", delimiter=',')


In [ ]:
print(df1.head)


In [ ]:
# Convert 'public_date' to datetime and sort data
df1['public_date'] = pd.to_datetime(df1['public_date'])
df1 = df1.sort_values(by=['permno', 'public_date'])

# Get unique company counts before filtering
initial_bankrupt_count = df1[df1['Bankrupt']].drop_duplicates('permno').shape[0]
initial_non_bankrupt_count = df1[~df1['Bankrupt']].drop_duplicates('permno').shape[0]

# Specify the periods, adjusted for zero-based index
periods = [x - 1 for x in [3, 6, 9, 12, 15, 18, 21, 24, 27, 30, 33, 36]]

# Filter rows for each 'permno' based on the specified periods
filtered_df1 = df1.groupby('permno').nth(periods).reset_index()

# Count unique bankrupt and non-bankrupt companies post-filtering
filtered_bankrupt_count = filtered_df1[filtered_df1['Bankrupt']].drop_duplicates('permno').shape[0]
filtered_non_bankrupt_count = filtered_df1[~filtered_df1['Bankrupt']].drop_duplicates('permno').shape[0]

# Print the initial and post-filtering counts
print(f"Initial Number of bankrupt companies: {initial_bankrupt_count}")
print(f"Initial Number of non-bankrupt companies: {initial_non_bankrupt_count}")
print(f"Filtered Number of bankrupt companies: {filtered_bankrupt_count}")
print(f"Filtered Number of non-bankrupt companies: {filtered_non_bankrupt_count}")

In [ ]:
# Ratios
ratios = ['X1', 'X2', 'X3', 'X4', 'X5', 'X6', 'X7', 'X8', 'X9', 'X10',
          'X11', 'X12', 'X13', 'X14', 'X15', 'X16', 'X17', 'X18', 'X19',
          'X20', 'X21', 'X22', 'X23', 'X24', 'X25', 'X26', 'X27', 'X28',
          'X29', 'X30', 'X31', 'X32', 'X33', 'X34', 'X35', 'X36', 'X37',
          'X38', 'X39', 'X40', 'X41', 'X42', 'X43', 'X44', 'X45', 'X46',
          'X47', 'X48', 'X49', 'X50', 'X51', 'X52', 'X53', 'X54', 'X55',
          'X56', 'X57', 'X58', 'X59', 'X60', 'X61', 'X62', 'X63', 'X64',
          'X65', 'X66', 'X67', 'X68', 'X69', 'X70', 'X71']

In [ ]:
# Prepare the dataset for LSTM
X, y = [], []
grouped = filtered_df1.groupby('permno')
sequence_length = 12

for _, group in grouped:
    group = group.sort_values(by='public_date')
    if len(group) >= sequence_length:
        X.append(group[ratios].tail(sequence_length).values)
        y.append(group['Bankruptcy'].iloc[-1])

X, y = np.array(X), np.array(y)

In [ ]:
# Split the dataset based on the date ranges
train_mask = (filtered_df1['public_date'] >= '1970-01-01') & (filtered_df1['public_date'] <= '2010-12-31')
test_mask = (filtered_df1['public_date'] >= '1970-01-01') & (filtered_df1['public_date'] <= '2020-12-31')
out_sample_mask = (filtered_df1['public_date'] >= '2011-01-01') & (filtered_df1['public_date'] <= '2020-12-31')

In [ ]:
X_train = np.array([X[i] for i in range(len(X)) if train_mask.iloc[i]])
y_train = np.array([y[i] for i in range(len(y)) if train_mask.iloc[i]])
X_test = np.array([X[i] for i in range(len(X)) if test_mask.iloc[i]])
y_test = np.array([y[i] for i in range(len(y)) if test_mask.iloc[i]])
X_out = np.array([X[i] for i in range(len(X)) if out_sample_mask.iloc[i]])
y_out = np.array([y[i] for i in range(len(y)) if out_sample_mask.iloc[i]])

# Count unique bankrupt and non-bankrupt companies in the training set
train_bankrupt_count = filtered_df1[train_mask & filtered_df1['Bankrupt']].drop_duplicates('permno').shape[0]
train_non_bankrupt_count = filtered_df1[train_mask & ~filtered_df1['Bankrupt']].drop_duplicates('permno').shape[0]

# Count unique bankrupt and non-bankrupt companies in the testing set
test_bankrupt_count = filtered_df1[test_mask & filtered_df1['Bankrupt']].drop_duplicates('permno').shape[0]
test_non_bankrupt_count = filtered_df1[test_mask & ~filtered_df1['Bankrupt']].drop_duplicates('permno').shape[0]

# Print the counts
print(f"Training data shape: {X_train.shape}, {y_train.shape}")
print(f"Testing data shape: {X_test.shape}, {y_test.shape}")
print(f"Training bankrupt companies: {train_bankrupt_count}")
print(f"Training non-bankrupt companies: {train_non_bankrupt_count}")
print(f"Testing bankrupt companies: {test_bankrupt_count}")
print(f"Testing non-bankrupt companies: {test_non_bankrupt_count}")

In [ ]:
# Impute missing values
imputer = SimpleImputer(strategy='mean')
X_train = imputer.fit_transform(X_train.reshape(-1, X_train.shape[-1])).reshape(X_train.shape)
X_test = imputer.transform(X_test.reshape(-1, X_test.shape[-1])).reshape(X_test.shape)
X_out = imputer.transform(X_out.reshape(-1, X_out.shape[-1])).reshape(X_out.shape)

In [ ]:
# Normalize the features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train.reshape(-1, X_train.shape[-1])).reshape(X_train.shape)
X_test_scaled = scaler.transform(X_test.reshape(-1, X_test.shape[-1])).reshape(X_test.shape)
X_out_scaled = scaler.transform(X_out.reshape(-1, X_out.shape[-1])).reshape(X_out.shape)

In [ ]:
# Balance the dataset using SMOTE
X_train_flattened = X_train_scaled.reshape(X_train_scaled.shape[0], -1)
sm = SMOTE(random_state=42)
X_train_resampled, y_train_resampled = sm.fit_resample(X_train_flattened, y_train)
X_train_resampled = X_train_resampled.reshape(-1, sequence_length, X_train_scaled.shape[2])

In [ ]:
# KFold and EarlyStopping setup
n_splits = 5
kf = KFold(n_splits=n_splits, shuffle=True, random_state=42)
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

In [ ]:
# Lists to store metrics
avg_accuracy = []
avg_recall = []
avg_precision = []
avg_f1 = []
avg_roc_auc = []
avg_kappa = []
avg_type_ii_error = []

In [ ]:
# Start KFold training
fold_var = 1
for train_index, val_index in kf.split(X_train_resampled):
    X_train_fold, X_val_fold = X_train_resampled[train_index], X_train_resampled[val_index]
    y_train_fold, y_val_fold = y_train_resampled[train_index], y_train_resampled[val_index]
    
    # Build the LSTM model
    model = Sequential([
        LSTM(50, input_shape=(sequence_length, len(ratios)), activation='tanh'),
        Dense(1, activation='sigmoid')
    ])
    model.compile(optimizer=Adam(learning_rate=0.01), loss='binary_crossentropy', metrics=['accuracy'])

    # Train the model
    history = model.fit(X_train_fold, y_train_fold, validation_data=(X_val_fold, y_val_fold),
                        epochs=20, batch_size=32, callbacks=[early_stopping])

    # Predictions for this fold
    y_pred = model.predict(X_val_fold)
    y_pred_binary = (y_pred > 0.5).astype(int)

    # Calculate metrics for this fold
    accuracy = accuracy_score(y_val_fold, y_pred_binary)
    recall = recall_score(y_val_fold, y_pred_binary)
    precision = precision_score(y_val_fold, y_pred_binary)
    f1 = f1_score(y_val_fold, y_pred_binary)
    roc_auc = roc_auc_score(y_val_fold, y_pred)
    kappa = cohen_kappa_score(y_val_fold, y_pred_binary)
    type_ii_error = 1 - recall

    print(f'Fold {fold_var}:')
    print(f'  Accuracy: {accuracy:.4f}')
    print(f'  Recall: {recall:.4f}')
    print(f'  Precision: {precision:.4f}')
    print(f'  F1 Score: {f1:.4f}')
    print(f'  ROC AUC: {roc_auc:.4f}')
    print(f'  Kappa: {kappa:.4f}')
    print(f'  Type II Error: {type_ii_error:.4f}')
    print()

    # Append metrics for this fold to the lists
    avg_accuracy.append(accuracy)
    avg_recall.append(recall)
    avg_precision.append(precision)
    avg_f1.append(f1)
    avg_roc_auc.append(roc_auc)
    avg_kappa.append(kappa)
    avg_type_ii_error.append(type_ii_error)

    fold_var += 1


In [ ]:
# Calculate average of all folds
print('Average metrics across all folds:')
print(f'  Average Accuracy: {np.mean(avg_accuracy):.4f}')
print(f'  Average Recall: {np.mean(avg_recall):.4f}')
print(f'  Average Precision: {np.mean(avg_precision):.4f}')
print(f'  Average F1 Score: {np.mean(avg_f1):.4f}')
print(f'  Average ROC AUC: {np.mean(avg_roc_auc):.4f}')
print(f'  Average Kappa: {np.mean(avg_kappa):.4f}')
print(f'  Average Type II Error: {np.mean(avg_type_ii_error):.4f}')

In [ ]:
# Predict on the train set
y_train_pred_probs = model.predict(X_train_scaled)
y_train_pred = (y_train_pred_probs > 0.5).astype(int)


In [ ]:
# Calculate metrics
train_accuracy = accuracy_score(y_train, y_train_pred)
train_precision = precision_score(y_train, y_train_pred)
train_recall = recall_score(y_train, y_train_pred)
train_f1 = f1_score(y_train, y_train_pred)
train_roc_auc = roc_auc_score(y_train, y_train_pred_probs)

# Print the metrics
print(f"Train Accuracy: {train_accuracy:.4f}")
print(f"Train Precision: {train_precision:.4f}")
print(f"Train Recall: {train_recall:.4f}")
print(f"Train F1 Score: {train_f1:.4f}")
print(f"Train ROC AUC: {train_roc_auc:.4f}")


In [ ]:
# Compute and print the confusion matrix
cm = confusion_matrix(y_train, y_train_pred)
print("Confusion Matrix:")
print(cm)


In [ ]:
# Predict on the test set
y_test_pred_probs = model.predict(X_test_scaled)
y_test_pred = (y_test_pred_probs > 0.5).astype(int)

In [ ]:
# Calculate metrics
test_accuracy = accuracy_score(y_test, y_test_pred)
test_precision = precision_score(y_test, y_test_pred)
test_recall = recall_score(y_test, y_test_pred)
test_f1 = f1_score(y_test, y_test_pred)
test_roc_auc = roc_auc_score(y_test, y_test_pred_probs)

# Print the metrics
print(f"Test Accuracy: {test_accuracy:.4f}")
print(f"Test Precision: {test_precision:.4f}")
print(f"Test Recall: {test_recall:.4f}")
print(f"Test F1 Score: {test_f1:.4f}")
print(f"Test ROC AUC: {test_roc_auc:.4f}")


In [ ]:
# Compute and print the confusion matrix
cm = confusion_matrix(y_test, y_test_pred)
print("Confusion Matrix:")
print(cm)

In [ ]:
# Predict on the Out of Sample set
y_out_pred_probs = model.predict(X_out_scaled)
y_out_pred = (y_out_pred_probs > 0.5).astype(int)

In [ ]:
# Calculate metrics
Out_accuracy = accuracy_score(y_out, y_out_pred)
Out_precision = precision_score(y_out, y_out_pred)
Out_recall = recall_score(y_out, y_out_pred)
Out_f1 = f1_score(y_out, y_out_pred)
Out_roc_auc = roc_auc_score(y_out, y_out_pred_probs)

# Print the metrics
print(f"Out of Sample Accuracy: {Out_accuracy:.4f}")
print(f"Out of Sample Precision: {Out_precision:.4f}")
print(f"Out of Sample Recall: {Out_recall:.4f}")
print(f"Out of Sample F1 Score: {Out_f1:.4f}")
print(f"Out of Sample ROC AUC: {Out_roc_auc:.4f}")


In [ ]:
# Compute and print the confusion matrix
cm = confusion_matrix(y_out, y_out_pred)
print("Confusion Matrix:")
print(cm)